In [24]:
# Import the required modules
import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np
import pickle as pkl

# Pre-Processing
from sklearn.preprocessing import StandardScaler, OneHotEncoder, OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer

# Models
from sklearn.neighbors import NearestNeighbors

# suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_csv("anime_ml.csv")
print(df.shape)
df.head()

(11903, 49)


,anime_id,name,episodes,rating,members,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Fantasy,Game,Historical,Horror,Josei,Kids,Magic,MartialArts,Mecha,Military,Music,Mystery,Parody,Police,Psychological,Romance,Samurai,School,Sci-Fi,Seinen,Shoujo,ShoujoAi,Shounen,ShounenAi,SliceofLife,Space,Sports,SuperPower,Supernatural,Thriller,Vampire,Movie,Music.1,ONA,OVA,Special,TV
0,32281,Kimi no Na wa.,-0.243330,2.840061,3.299624,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0
1,5114,Fullmetal Alchemist: Brotherhood,1.098297,2.731791,14.040307,1,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
2,28977,Gintama,0.821453,2.721948,1.735380,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1
3,9253,Steins Gate,0.246470,2.643206,11.865257,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1
4,9969,Gintama',0.821453,2.633363,2.405574,1,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1


In [3]:
df.describe()

,anime_id,episodes,rating,members,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Fantasy,Game,Historical,Horror,Josei,Kids,Magic,MartialArts,Mecha,Military,Music,Mystery,Parody,Police,Psychological,Romance,Samurai,School,Sci-Fi,Seinen,Shoujo,ShoujoAi,Shounen,ShounenAi,SliceofLife,Space,Sports,SuperPower,Supernatural,Thriller,Vampire,Movie,Music.1,ONA,OVA,Special,TV
count,11903.000000,1.190300e+04,1.190300e+04,1.190300e+04,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000,11903.000000
mean,13515.515080,1.910222e-17,1.910222e-16,3.820443e-17,0.233555,0.193901,0.005965,0.378728,0.019995,0.024364,0.166765,0.189028,0.014618,0.066454,0.030497,0.004369,0.130975,0.063849,0.021675,0.078216,0.035033,0.070234,0.040830,0.033437,0.016382,0.019071,0.121314,0.012182,0.098715,0.171049,0.044695,0.049567,0.004537,0.141057,0.005377,0.099555,0.031589,0.044611,0.037554,0.084601,0.007225,0.008485,0.193397,0.040914,0.051164,0.273881,0.140217,0.300428
std,11174.446066,1.000042e+00,1.000042e+00,1.000042e+00,0.423110,0.395369,0.077005,0.485091,0.139989,0.154182,0.372781,0.391547,0.120024,0.249084,0.171956,0.065954,0.337388,0.244495,0.145627,0.268522,0.183871,0.255552,0.197905,0.179782,0.126946,0.136780,0.326505,0.109701,0.298291,0.376568,0.206641,0.217058,0.067205,0.348095,0.073132,0.299418,0.174910,0.206456,0.190122,0.278298,0.084696,0.091728,0.394978,0.198099,0.220341,0.445967,0.347226,0.458464
min,1.000000,-2.433302e-01,-4.738868e+00,-3.339717e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3361.500000,-2.433302e-01,-5.753782e-01,-3.298241e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,9885.000000,-2.220345e-01,8.408702e-02,-3.051564e-01,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,23485.000000,-9.077896e-03,6.844957e-01,-1.560273e-01,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,1.000000
max,34527.000000,3.845089e+01,3.460155e+00,1.802937e+01,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,

In [4]:
def make_recommendation(anime_length, anime_name):
    # Step 1: Load the dataset
    df = pd.read_csv("anime_ml.csv")
    
    # Remove any rows with missing values and reset the index
    df = df.dropna(how="any").reset_index(drop=True)

    # # Extract the release year from the album release date and convert to integer
    # df["track_release_year"] = df.track_album_release_date.apply(lambda x: int(x[0:4]))

    # # Drop unnecessary columns that are not needed for recommendation
    # df = df.drop(columns=["track_album_id", "track_album_name", "track_album_release_date", 
    #                       "playlist_name", "playlist_id"])
    # Update the Column name
    df.rename(columns={'Music.1': 'Music'}, inplace=True)
    
    # Remove duplicate anime based on track_id and reset index again
    df = df.drop_duplicates(subset=["name"]).reset_index(drop=True)

    # Step 2: Define the columns for metadata and features
    meta_cols = ["anime_id","name"]  # Metadata columns to keep anime info
    feature_cols = ['episodes', 'rating', 'members', 'Action', 'Adventure', 'Cars',
       'Comedy', 'Dementia', 'Demons', 'Drama', 'Fantasy', 'Game',
       'Historical', 'Horror', 'Josei', 'Kids', 'Magic', 'MartialArts',
       'Mecha', 'Military', 'Music', 'Mystery', 'Parody', 'Police',
       'Psychological', 'Romance', 'Samurai', 'School', 'Sci-Fi', 'Seinen',
       'Shoujo', 'ShoujoAi', 'Shounen', 'ShounenAi', 'SliceofLife', 'Space',
       'Sports', 'SuperPower', 'Supernatural', 'Thriller', 'Vampire', 'Movie',
       'Music', 'ONA', 'OVA', 'Special', 'TV']

    # Step 3: Define preprocessing steps for the data

    # Preprocessing for numeric features: fill missing values with the mean and standardize the values
    numeric_features = []
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),  # Fill missing values with the mean
        ('scaler', StandardScaler())])  # Standardize features

    # Preprocessing for binary features (if any): fill missing values with most frequent and label encode
    binary_features = []
    binary_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent', missing_values=pd.NA)),  # Fill missing values
        ('label', OrdinalEncoder())])  # Convert binary features to ordinal values

    # Preprocessing for categorical features: fill missing values and apply one-hot encoding
    categorical_features = []
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent', missing_values=pd.NA)),  # Fill missing values
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])  # Apply one-hot encoding

    # Combine all preprocessing steps into a single preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),  # Numeric features preprocessing
            ('binary', binary_transformer, binary_features),  # Binary features preprocessing (if any)
            ('cat', categorical_transformer, categorical_features)])  # Categorical features preprocessing

    # Step 4: Get the target track's ID based on track name and artist
    # Select the most popular track by track_name and track_artist combination
    anime_id = df.loc[(df.name == anime_name)] \
                .sort_values(by="rating", ascending=False).anime_id.values[0]
    # anime_id = 22319
    # Step 5: Prepare the feature matrix for the Nearest Neighbors model
    X = df.loc[:, feature_cols]  # Select the feature columns

    # Apply preprocessing to the feature matrix
    # preprocessor.fit(X)  # Fit the preprocessor to the feature matrix
    # X_preprocessed = preprocessor.transform(X)  # Transform the feature matrix

    # Step 6: Initialize the Nearest Neighbors model
    # # define the number of nearest neighbors to consider and use cosine similarity as the metric
    k = anime_length
    model1 = NearestNeighbors(n_neighbors=k, metric="cosine")

    # Fit the Nearest Neighbors model to the preprocessed data
    # model1.fit(X_preprocessed)
    model1.fit(X)
    # Step 7: Extract features of the anime
    anime_features = df.loc[df.anime_id == anime_id, feature_cols]  # Get the feature vector for the target anime
    # anime_features_preprocessed = preprocessor.transform(anime_features)  # Preprocess the target track features

    # Step 8: Find the nearest neighbors (anime most similar to the target selection)
    # distances, indices = model1.kneighbors(anime_features_preprocessed)  # Get distances and indices of neighbors
    distances, indices = model1.kneighbors(anime_features)  # Get distances and indices of neighbors

    # Step 9: Retrieve the metadata of the recommended tracks
    animes = df.iloc[indices[0]]  # Select anime corresponding to the nearest neighbors
    animes["distance"] = distances[0]  # Add the distance of each neighbor as a new column

    # Step 10: Filter the columns for the final output
    cols = animes.columns # you can explicitly choose to return specific columns here
    animes = animes.loc[:, cols]  # Keep the relevant columns
    animes = animes.sort_values(by="distance")  # Sort the tracks by their distance (most similar first)

    # Step 11: Return the recommended tracks as a list of dictionaries
    return animes.to_dict(orient="records")

In [5]:
 # User input
anime_length = 10
anime_name = "Naruto"


response = make_recommendation(anime_length, anime_name) # used for the flask app, returns JSON

# to test
pd.DataFrame(response)

,anime_id,name,episodes,rating,members,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Fantasy,Game,Historical,Horror,Josei,Kids,Magic,MartialArts,Mecha,Military,Music,Mystery,Parody,Police,Psychological,Romance,Samurai,School,Sci-Fi,Seinen,Shoujo,ShoujoAi,Shounen,ShounenAi,SliceofLife,Space,Sports,SuperPower,Supernatural,Thriller,Vampire,Movie,ONA,OVA,Special,TV,distance
0,20,Naruto,4.420420,1.304590,12.041390,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0.000000
1,6702,Fairy Tail,3.462115,1.708143,10.253670,1,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0.019756
2,269,Bleach,7.529587,1.442389,10.968436,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,1,0.035303
3,11061,Hunter x Hunter (2011),2.887132,2.603835,7.378760,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0.038068
4,1482,D.Gray-man,1.928827,1.688458,5.722366,1,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0.042725
5,3588,Soul Eater,0.821453,1.570345,10.173871,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,0.055189
6,223,Dragon Ball,2.993610,1.649087,5.390982,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0.059360
7,5114,Fullmetal Alchemist: Brotherhood,1.098297,2.731791,14.040307,1,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0.059473
8,121,Fullmetal Alchemist,0.821453,1.816414,10.539721,1,1,0,1,0,0,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0.063370
9,16498,Shingeki no Kyojin,0.267766,2.023112,15.897883,1,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0.063634


In [6]:
def make_recommendation_euc(anime_length, anime_name):
    # Step 1: Load the dataset
    df = pd.read_csv("anime_ml.csv")
    
    # Remove any rows with missing values and reset the index
    df = df.dropna(how="any").reset_index(drop=True)

    # # Extract the release year from the album release date and convert to integer
    # df["track_release_year"] = df.track_album_release_date.apply(lambda x: int(x[0:4]))

    # # Drop unnecessary columns that are not needed for recommendation
    # df = df.drop(columns=["track_album_id", "track_album_name", "track_album_release_date", 
    #                       "playlist_name", "playlist_id"])
    # Update the Column name
    df.rename(columns={'Music.1': 'Music'}, inplace=True)
    
    # Remove duplicate anime based on track_id and reset index again
    df = df.drop_duplicates(subset=["name"]).reset_index(drop=True)

    # Step 2: Define the columns for metadata and features
    meta_cols = ["anime_id","name"]  # Metadata columns to keep anime info
    feature_cols = ['episodes', 'rating', 'members', 'Action', 'Adventure', 'Cars',
       'Comedy', 'Dementia', 'Demons', 'Drama', 'Fantasy', 'Game',
       'Historical', 'Horror', 'Josei', 'Kids', 'Magic', 'MartialArts',
       'Mecha', 'Military', 'Music', 'Mystery', 'Parody', 'Police',
       'Psychological', 'Romance', 'Samurai', 'School', 'Sci-Fi', 'Seinen',
       'Shoujo', 'ShoujoAi', 'Shounen', 'ShounenAi', 'SliceofLife', 'Space',
       'Sports', 'SuperPower', 'Supernatural', 'Thriller', 'Vampire', 'Movie',
       'Music', 'ONA', 'OVA', 'Special', 'TV']

    # Step 3: Define preprocessing steps for the data

    # Preprocessing for numeric features: fill missing values with the mean and standardize the values
    numeric_features = []
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),  # Fill missing values with the mean
        ('scaler', StandardScaler())])  # Standardize features

    # Preprocessing for binary features (if any): fill missing values with most frequent and label encode
    binary_features = []
    binary_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent', missing_values=pd.NA)),  # Fill missing values
        ('label', OrdinalEncoder())])  # Convert binary features to ordinal values

    # Preprocessing for categorical features: fill missing values and apply one-hot encoding
    categorical_features = []
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent', missing_values=pd.NA)),  # Fill missing values
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])  # Apply one-hot encoding

    # Combine all preprocessing steps into a single preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),  # Numeric features preprocessing
            ('binary', binary_transformer, binary_features),  # Binary features preprocessing (if any)
            ('cat', categorical_transformer, categorical_features)])  # Categorical features preprocessing

    # Step 4: Get the target track's ID based on track name and artist
    # Select the most popular track by track_name and track_artist combination
    anime_id = df.loc[(df.name == anime_name)] \
                .sort_values(by="rating", ascending=False).anime_id.values[0]
    # anime_id = 22319
    # Step 5: Prepare the feature matrix for the Nearest Neighbors model
    X = df.loc[:, feature_cols]  # Select the feature columns

    # Apply preprocessing to the feature matrix
    # preprocessor.fit(X)  # Fit the preprocessor to the feature matrix
    # X_preprocessed = preprocessor.transform(X)  # Transform the feature matrix

    # Step 6: Initialize the Nearest Neighbors model
    # # define the number of nearest neighbors to consider and use cosine similarity as the metric
    k = anime_length
    model1 = NearestNeighbors(n_neighbors=k, metric="euclidean")

    # Fit the Nearest Neighbors model to the preprocessed data
    # model1.fit(X_preprocessed)
    model1.fit(X)
    # Step 7: Extract features of the anime
    anime_features = df.loc[df.anime_id == anime_id, feature_cols]  # Get the feature vector for the target anime
    # anime_features_preprocessed = preprocessor.transform(anime_features)  # Preprocess the target track features

    # Step 8: Find the nearest neighbors (anime most similar to the target selection)
    # distances, indices = model1.kneighbors(anime_features_preprocessed)  # Get distances and indices of neighbors
    distances, indices = model1.kneighbors(anime_features)  # Get distances and indices of neighbors

    # Step 9: Retrieve the metadata of the recommended tracks
    animes = df.iloc[indices[0]]  # Select anime corresponding to the nearest neighbors
    animes["distance"] = distances[0]  # Add the distance of each neighbor as a new column

    # Step 10: Filter the columns for the final output
    cols = animes.columns # you can explicitly choose to return specific columns here
    animes = animes.loc[:, cols]  # Keep the relevant columns
    animes = animes.sort_values(by="distance")  # Sort the tracks by their distance (most similar first)

    # Step 11: Return the recommended tracks as a list of dictionaries
    return animes.to_dict(orient="records")

In [7]:
 # User input
anime_length = 10
anime_name = "Naruto"


response = make_recommendation_euc(anime_length, anime_name) # used for the flask app, returns JSON

# to test
pd.DataFrame(response)

,anime_id,name,episodes,rating,members,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Fantasy,Game,Historical,Horror,Josei,Kids,Magic,MartialArts,Mecha,Military,Music,Mystery,Parody,Police,Psychological,Romance,Samurai,School,Sci-Fi,Seinen,Shoujo,ShoujoAi,Shounen,ShounenAi,SliceofLife,Space,Sports,SuperPower,Supernatural,Thriller,Vampire,Movie,ONA,OVA,Special,TV,distance
0,20,Naruto,4.420420,1.304590,12.041390,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0.000000
1,6702,Fairy Tail,3.462115,1.708143,10.253670,1,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,3.045841
2,269,Bleach,7.529587,1.442389,10.968436,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,1,3.582895
3,3588,Soul Eater,0.821453,1.570345,10.173871,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,4.637975
4,121,Fullmetal Alchemist,0.821453,1.816414,10.539721,1,1,0,1,0,0,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,4.740204
5,10620,Mirai Nikki (TV),0.289061,1.560502,11.568556,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,4.935300
6,5114,Fullmetal Alchemist: Brotherhood,1.098297,2.731791,14.040307,1,1,0,0,0,0,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,5.006903
7,4224,Toradora!,0.267766,1.934527,11.145239,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,5.044249
8,1575,Code Geass: Hangyaku no Lelouch,0.267766,2.308552,12.618310,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,1,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,5.058193
9,6547,Angel Beats!,0.012218,1.875470,12.666215,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,5.113566


In [8]:
def make_recommendation_man(anime_length, anime_name):
    # Step 1: Load the dataset
    df = pd.read_csv("anime_ml.csv")
    
    # Remove any rows with missing values and reset the index
    df = df.dropna(how="any").reset_index(drop=True)

    # # Extract the release year from the album release date and convert to integer
    # df["track_release_year"] = df.track_album_release_date.apply(lambda x: int(x[0:4]))

    # # Drop unnecessary columns that are not needed for recommendation
    # df = df.drop(columns=["track_album_id", "track_album_name", "track_album_release_date", 
    #                       "playlist_name", "playlist_id"])
    # Update the Column name
    df.rename(columns={'Music.1': 'Music'}, inplace=True)
    
    # Remove duplicate anime based on track_id and reset index again
    df = df.drop_duplicates(subset=["name"]).reset_index(drop=True)

    # Step 2: Define the columns for metadata and features
    meta_cols = ["anime_id","name"]  # Metadata columns to keep anime info
    feature_cols = ['episodes', 'rating', 'members', 'Action', 'Adventure', 'Cars',
       'Comedy', 'Dementia', 'Demons', 'Drama', 'Fantasy', 'Game',
       'Historical', 'Horror', 'Josei', 'Kids', 'Magic', 'MartialArts',
       'Mecha', 'Military', 'Music', 'Mystery', 'Parody', 'Police',
       'Psychological', 'Romance', 'Samurai', 'School', 'Sci-Fi', 'Seinen',
       'Shoujo', 'ShoujoAi', 'Shounen', 'ShounenAi', 'SliceofLife', 'Space',
       'Sports', 'SuperPower', 'Supernatural', 'Thriller', 'Vampire', 'Movie',
       'Music', 'ONA', 'OVA', 'Special', 'TV']

    # Step 3: Define preprocessing steps for the data

    # Preprocessing for numeric features: fill missing values with the mean and standardize the values
    numeric_features = []
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),  # Fill missing values with the mean
        ('scaler', StandardScaler())])  # Standardize features

    # Preprocessing for binary features (if any): fill missing values with most frequent and label encode
    binary_features = []
    binary_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent', missing_values=pd.NA)),  # Fill missing values
        ('label', OrdinalEncoder())])  # Convert binary features to ordinal values

    # Preprocessing for categorical features: fill missing values and apply one-hot encoding
    categorical_features = []
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent', missing_values=pd.NA)),  # Fill missing values
        ('onehot', OneHotEncoder(handle_unknown='ignore'))])  # Apply one-hot encoding

    # Combine all preprocessing steps into a single preprocessor
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, numeric_features),  # Numeric features preprocessing
            ('binary', binary_transformer, binary_features),  # Binary features preprocessing (if any)
            ('cat', categorical_transformer, categorical_features)])  # Categorical features preprocessing

    # Step 4: Get the target track's ID based on track name and artist
    # Select the most popular track by track_name and track_artist combination
    anime_id = df.loc[(df.name == anime_name)] \
                .sort_values(by="rating", ascending=False).anime_id.values[0]
    # anime_id = 22319
    # Step 5: Prepare the feature matrix for the Nearest Neighbors model
    X = df.loc[:, feature_cols]  # Select the feature columns

    # Apply preprocessing to the feature matrix
    # preprocessor.fit(X)  # Fit the preprocessor to the feature matrix
    # X_preprocessed = preprocessor.transform(X)  # Transform the feature matrix

    # Step 6: Initialize the Nearest Neighbors model
    # # define the number of nearest neighbors to consider and use cosine similarity as the metric
    k = anime_length
    model1 = NearestNeighbors(n_neighbors=k, metric="manhattan")

    # Fit the Nearest Neighbors model to the preprocessed data
    # model1.fit(X_preprocessed)
    model1.fit(X)
    # Step 7: Extract features of the anime
    anime_features = df.loc[df.anime_id == anime_id, feature_cols]  # Get the feature vector for the target anime
    # anime_features_preprocessed = preprocessor.transform(anime_features)  # Preprocess the target track features

    # Step 8: Find the nearest neighbors (anime most similar to the target selection)
    # distances, indices = model1.kneighbors(anime_features_preprocessed)  # Get distances and indices of neighbors
    distances, indices = model1.kneighbors(anime_features)  # Get distances and indices of neighbors

    # Step 9: Retrieve the metadata of the recommended tracks
    animes = df.iloc[indices[0]]  # Select anime corresponding to the nearest neighbors
    animes["distance"] = distances[0]  # Add the distance of each neighbor as a new column

    # Step 10: Filter the columns for the final output
    cols = animes.columns # you can explicitly choose to return specific columns here
    animes = animes.loc[:, cols]  # Keep the relevant columns
    animes = animes.sort_values(by="distance")  # Sort the tracks by their distance (most similar first)

    # Step 11: Return the recommended tracks as a list of dictionaries
    return animes.to_dict(orient="records")

In [9]:
 # User input
anime_length = 10
anime_name = "Naruto"


response = make_recommendation_man(anime_length, anime_name) # used for the flask app, returns JSON

# to test
pd.DataFrame(response)

,anime_id,name,episodes,rating,members,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Fantasy,Game,Historical,Horror,Josei,Kids,Magic,MartialArts,Mecha,Military,Music,Mystery,Parody,Police,Psychological,Romance,Samurai,School,Sci-Fi,Seinen,Shoujo,ShoujoAi,Shounen,ShounenAi,SliceofLife,Space,Sports,SuperPower,Supernatural,Thriller,Vampire,Movie,ONA,OVA,Special,TV,distance
0,20,Naruto,4.420420,1.304590,12.041390,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0.000000
1,269,Bleach,7.529587,1.442389,10.968436,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,1,6.319920
2,6702,Fairy Tail,3.462115,1.708143,10.253670,1,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,8.149578
3,813,Dragon Ball Z,5.932412,1.806571,6.469695,1,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,9.585668
4,1604,Katekyo Hitman Reborn!,4.058394,1.855785,4.340540,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,9.614072
5,11061,Hunter x Hunter (2011),2.887132,2.603835,7.378760,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,10.495163
6,3588,Soul Eater,0.821453,1.570345,10.173871,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,10.732240
7,18679,Kill la Kill,0.246470,1.717986,8.868657,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,10.760080
8,223,Dragon Ball,2.993610,1.649087,5.390982,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,11.421715
9,6547,Angel Beats!,0.012218,1.875470,12.666215,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,11.603907


In [10]:
def make_recommendation(anime_length, anime_name, genres):
    # Step 1: Load the dataset
    df = pd.read_csv("anime_ml.csv")
    
    # Remove any rows with missing values and reset the index
    df = df.dropna(how="any").reset_index(drop=True)

    # Update the Column name
    df.rename(columns={'Music.1': 'Music'}, inplace=True)
    
    # Remove duplicate anime based on name
    df = df.drop_duplicates(subset=["name"]).reset_index(drop=True)

    # Define the columns for metadata and features
    meta_cols = ["anime_id", "name"]
    feature_cols = ['episodes', 'rating', 'members', 'Action', 'Adventure', 'Cars',
                    'Comedy', 'Dementia', 'Demons', 'Drama', 'Fantasy', 'Game',
                    'Historical', 'Horror', 'Josei', 'Kids', 'Magic', 'MartialArts',
                    'Mecha', 'Military', 'Music', 'Mystery', 'Parody', 'Police',
                    'Psychological', 'Romance', 'Samurai', 'School', 'Sci-Fi', 'Seinen',
                    'Shoujo', 'ShoujoAi', 'Shounen', 'ShounenAi', 'SliceofLife', 'Space',
                    'Sports', 'SuperPower', 'Supernatural', 'Thriller', 'Vampire', 'Movie',
                    'Music', 'ONA', 'OVA', 'Special', 'TV']

    # Preprocessing for numeric features
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])
    
    # Categorical features
    categorical_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='most_frequent')),
        ('onehot', OneHotEncoder(handle_unknown='ignore'))
    ])

    # Combine all preprocessing steps
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, ['episodes', 'rating', 'members']),
            ('cat', categorical_transformer, feature_cols[3:])  # Adjust as necessary
        ]
    )

    # Step 4: Get the target anime's ID based on name
    anime_match = df.loc[df.name == anime_name]
    if anime_match.empty:
        raise ValueError(f"Anime '{anime_name}' not found.")
    
    anime_id = anime_match.sort_values(by="rating", ascending=False).anime_id.values[0]

    # Step 5: Filter for the specified genres
    genre_filter = df[genres].sum(axis=1) > 0  # Keep rows where at least one genre is present
    filtered_df = df[genre_filter]

    # If no anime matches the genres, raise an error
    if filtered_df.empty:
        raise ValueError(f"No anime found for genres: {', '.join(genres)}")

    # Prepare the feature matrix for the Nearest Neighbors model
    X = filtered_df[feature_cols]  # Select the feature columns
    X_preprocessed = preprocessor.fit_transform(X)  # Fit and transform the feature matrix

    # Step 6: Initialize and fit the Nearest Neighbors model
    k = anime_length
    model1 = NearestNeighbors(n_neighbors=k, metric="cosine")
    model1.fit(X_preprocessed)

    # Step 7: Extract features of the selected anime
    anime_features = df.loc[df.anime_id == anime_id, feature_cols]
    anime_features_preprocessed = preprocessor.transform(anime_features)

    # Step 8: Find the nearest neighbors
    distances, indices = model1.kneighbors(anime_features_preprocessed)

    # Step 9: Retrieve the metadata of the recommended animes
    animes = filtered_df.iloc[indices[0]]
    animes["distance"] = distances[0]  # Add distances to the DataFrame

    # Step 10: Filter the columns for the final output
    animes = animes.sort_values(by="distance")  # Sort by distance

    # Step 11: Return the recommended animes as a list of dictionaries
    return animes.to_dict(orient="records")

In [11]:
def make_recommendation(anime_length, anime_name, genres):
    # Load the dataset
    df = pd.read_csv("anime_ml.csv")

    # Remove any rows with missing values and reset the index
    df = df.dropna(how="any").reset_index(drop=True)

    # Update the column name if necessary
    df.rename(columns={'Music.1': 'Music'}, inplace=True)

    # Remove duplicate anime based on name
    df = df.drop_duplicates(subset=["name"]).reset_index(drop=True)

    # Define the columns for metadata and features
    meta_cols = ["anime_id", "name"]
    feature_cols = ['episodes', 'rating', 'members'] + genres  # Include boolean genre columns directly

    # Preprocessing for numeric features
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])
    
    # Combine all preprocessing steps
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, ['episodes', 'rating', 'members'])
        ]
    )

    # Get the target anime's ID based on name
    anime_match = df.loc[df.name == anime_name]
    if anime_match.empty:
        raise ValueError(f"Anime '{anime_name}' not found.")
    
    anime_id = anime_match.sort_values(by="rating", ascending=False).anime_id.values[0]

    # Filter for the specified genres
    genre_filter = df[genres].sum(axis=1) > 0  # Keep rows where at least one genre is present
    filtered_df = df[genre_filter]

    # If no anime matches the genres, raise an error
    if filtered_df.empty:
        raise ValueError(f"No anime found for genres: {', '.join(genres)}")

    # Prepare the feature matrix for the Nearest Neighbors model
    X = filtered_df[feature_cols]  # Select the feature columns
    X_preprocessed = preprocessor.fit_transform(X)  # Fit and transform the feature matrix

    # Initialize and fit the Nearest Neighbors model
    k = anime_length
    model1 = NearestNeighbors(n_neighbors=k, metric="cosine")
    model1.fit(X_preprocessed)

    # Extract features of the selected anime
    anime_features = df.loc[df.anime_id == anime_id, feature_cols]
    anime_features_preprocessed = preprocessor.transform(anime_features)

    # Find the nearest neighbors
    distances, indices = model1.kneighbors(anime_features_preprocessed)

    # Retrieve the metadata of the recommended animes
    animes = filtered_df.iloc[indices[0]]
    animes["distance"] = distances[0]  # Add distances to the DataFrame

    # Filter the columns for the final output
    animes = animes.sort_values(by="distance")  # Sort by distance

    # Return the recommended animes as a list of dictionaries
    return animes.to_dict(orient="records")


In [12]:
anime_length = 10
anime_name = "Naruto"
genres = ["MartialArts"]  # Specify the genres you want to filter by

response = make_recommendation(anime_length, anime_name, genres)

In [13]:
pd.DataFrame(response)

,anime_id,name,episodes,rating,members,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Fantasy,Game,Historical,Horror,Josei,Kids,Magic,MartialArts,Mecha,Military,Music,Mystery,Parody,Police,Psychological,Romance,Samurai,School,Sci-Fi,Seinen,Shoujo,ShoujoAi,Shounen,ShounenAi,SliceofLife,Space,Sports,SuperPower,Supernatural,Thriller,Vampire,Movie,ONA,OVA,Special,TV,distance
0,20,Naruto,4.420420,1.304590,12.041390,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0.000000
1,223,Dragon Ball,2.993610,1.649087,5.390982,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0.027687
2,174,Tenjou Tenge,0.246470,0.605754,1.539542,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0.040721
3,9367,Freezing,-0.009078,0.704181,2.550592,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.077985
4,813,Dragon Ball Z,5.932412,1.806571,6.469695,1,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0.081486
5,10213,Maji de Watashi ni Koi Shinasai!,-0.009078,0.527011,1.779228,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0.082897
6,4999,Asu no Yoichi!,-0.009078,0.527011,1.671737,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0.085265
7,11761,Medaka Box,-0.009078,0.714024,1.658950,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0.104781
8,9936,Maken-Ki!,-0.009078,0.202200,1.552818,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0.106505
9,10396,Ben-To,-0.009078,0.989621,2.320523,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0.109306


In [14]:
def make_recommendation(anime_length, anime_name, genres):
    # Load the dataset
    df = pd.read_csv("anime_ml.csv")

    # Remove any rows with missing values and reset the index
    df = df.dropna(how="any").reset_index(drop=True)

    # Update the column name if necessary
    df.rename(columns={'Music.1': 'Music'}, inplace=True)

    # Remove duplicate anime based on name
    df = df.drop_duplicates(subset=["name"]).reset_index(drop=True)

    # Define the columns for metadata and features
    meta_cols = ["anime_id", "name"]
    feature_cols = ['episodes', 'rating', 'members'] + genres  # Include boolean genre columns directly

    # Preprocessing for numeric features
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])
    
    # Combine all preprocessing steps
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, ['episodes', 'rating', 'members'])
        ]
    )

    # Get the target anime's ID based on name
    anime_match = df.loc[df.name == anime_name]
    if anime_match.empty:
        raise ValueError(f"Anime '{anime_name}' not found.")
    
    anime_id = anime_match.sort_values(by="rating", ascending=False).anime_id.values[0]

    # Filter for the specified genres
    genre_filter = df[genres].sum(axis=1) > 0  # Keep rows where at least one genre is present
    filtered_df = df[genre_filter]

    # If no anime matches the genres, raise an error
    if filtered_df.empty:
        raise ValueError(f"No anime found for genres: {', '.join(genres)}")

    # Prepare the feature matrix for the Nearest Neighbors model
    X = filtered_df[feature_cols]  # Select the feature columns
    X_preprocessed = preprocessor.fit_transform(X)  # Fit and transform the feature matrix

    # Initialize and fit the Nearest Neighbors model
    k = anime_length
    model1 = NearestNeighbors(n_neighbors=k, metric="euclidean")
    model1.fit(X_preprocessed)

    # Extract features of the selected anime
    anime_features = df.loc[df.anime_id == anime_id, feature_cols]
    anime_features_preprocessed = preprocessor.transform(anime_features)

    # Find the nearest neighbors
    distances, indices = model1.kneighbors(anime_features_preprocessed)

    # Retrieve the metadata of the recommended animes
    animes = filtered_df.iloc[indices[0]]
    animes["distance"] = distances[0]  # Add distances to the DataFrame

    # Filter the columns for the final output
    animes = animes.sort_values(by="distance")  # Sort by distance

    # Return the recommended animes as a list of dictionaries
    return animes.to_dict(orient="records")

In [15]:
# User input
anime_length = 10
anime_name = "Naruto"
genres = ["MartialArts"]  # Specify the genres you want to filter by

response = make_recommendation(anime_length, anime_name, genres)

In [16]:
# To test
pd.DataFrame(response)

,anime_id,name,episodes,rating,members,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Fantasy,Game,Historical,Horror,Josei,Kids,Magic,MartialArts,Mecha,Military,Music,Mystery,Parody,Police,Psychological,Romance,Samurai,School,Sci-Fi,Seinen,Shoujo,ShoujoAi,Shounen,ShounenAi,SliceofLife,Space,Sports,SuperPower,Supernatural,Thriller,Vampire,Movie,ONA,OVA,Special,TV,distance
0,20,Naruto,4.420420,1.304590,12.041390,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0.000000
1,813,Dragon Ball Z,5.932412,1.806571,6.469695,1,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,5.261440
2,223,Dragon Ball,2.993610,1.649087,5.390982,0,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,6.174024
3,392,Yuu Yuu Hakusho,2.120488,1.954212,3.197965,1,0,0,1,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,8.310274
4,6594,Katanagatari,-0.009078,1.973898,3.419359,1,1,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,8.799445
5,9367,Freezing,-0.009078,0.704181,2.550592,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,9.498797
6,210,Ranma 1/2,3.163976,1.363646,1.571472,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,9.554136
7,6033,Dragon Ball Kai,1.801053,1.442389,1.781927,1,1,0,1,0,0,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,9.589744
8,10396,Ben-To,-0.009078,0.989621,2.320523,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,9.668844
9,1559,Shijou Saikyou no Deshi Kenichi,0.800157,1.737672,2.004334,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,9.670072


In [17]:
def make_recommendation(anime_length, anime_name, genres):
    # Load the dataset
    df = pd.read_csv("anime_ml.csv")

    # Remove any rows with missing values and reset the index
    df = df.dropna(how="any").reset_index(drop=True)

    # Update the column name if necessary
    df.rename(columns={'Music.1': 'Music'}, inplace=True)

    # Remove duplicate anime based on name
    df = df.drop_duplicates(subset=["name"]).reset_index(drop=True)

    # Define the columns for metadata and features
    meta_cols = ["anime_id", "name"]
    feature_cols = ['episodes', 'rating', 'members'] + genres  # Include boolean genre columns directly

    # Preprocessing for numeric features
    numeric_transformer = Pipeline(steps=[
        ('imputer', SimpleImputer(strategy='mean')),
        ('scaler', StandardScaler())
    ])
    
    # Combine all preprocessing steps
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', numeric_transformer, ['episodes', 'rating', 'members'])
        ]
    )

    # Get the target anime's ID based on name
    anime_match = df.loc[df.name == anime_name]
    if anime_match.empty:
        raise ValueError(f"Anime '{anime_name}' not found.")
    
    anime_id = anime_match.sort_values(by="rating", ascending=False).anime_id.values[0]

    # Filter for the specified genres
    genre_filter = df[genres].sum(axis=1) > 0  # Keep rows where at least one genre is present
    filtered_df = df[genre_filter]

    # If no anime matches the genres, raise an error
    if filtered_df.empty:
        raise ValueError(f"No anime found for genres: {', '.join(genres)}")

    # Prepare the feature matrix for the Nearest Neighbors model
    X = filtered_df[feature_cols]  # Select the feature columns
    X_preprocessed = preprocessor.fit_transform(X)  # Fit and transform the feature matrix

    # Initialize and fit the Nearest Neighbors model
    k = anime_length
    model1 = NearestNeighbors(n_neighbors=k, metric="manhattan")
    model1.fit(X_preprocessed)

    # Extract features of the selected anime
    anime_features = df.loc[df.anime_id == anime_id, feature_cols]
    anime_features_preprocessed = preprocessor.transform(anime_features)

    # Find the nearest neighbors
    distances, indices = model1.kneighbors(anime_features_preprocessed)

    # Retrieve the metadata of the recommended animes
    animes = filtered_df.iloc[indices[0]]
    animes["distance"] = distances[0]  # Add distances to the DataFrame

    # Filter the columns for the final output
    animes = animes.sort_values(by="distance")  # Sort by distance

    # Return the recommended animes as a list of dictionaries
    return animes.to_dict(orient="records")

In [28]:
anime_length = 10
anime_name = "Naruto"
genres = ["Action", "MartialArts", "Comedy"]  # Specify the genres you want to filter by

response = make_recommendation(anime_length, anime_name, genres) 

In [29]:
# To test
pd.DataFrame(response)

,anime_id,name,episodes,rating,members,Action,Adventure,Cars,Comedy,Dementia,Demons,Drama,Fantasy,Game,Historical,Horror,Josei,Kids,Magic,MartialArts,Mecha,Military,Music,Mystery,Parody,Police,Psychological,Romance,Samurai,School,Sci-Fi,Seinen,Shoujo,ShoujoAi,Shounen,ShounenAi,SliceofLife,Space,Sports,SuperPower,Supernatural,Thriller,Vampire,Movie,ONA,OVA,Special,TV,distance
0,20,Naruto,4.420420,1.304590,12.041390,1,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,1,0.000000
1,6702,Fairy Tail,3.462115,1.708143,10.253670,1,1,0,1,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,2.832016
2,269,Bleach,7.529587,1.442389,10.968436,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0,1,3.954879
3,10620,Mirai Nikki (TV),0.289061,1.560502,11.568556,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0,0,0,0,0,1,4.521678
4,226,Elfen Lied,0.012218,1.343961,10.958583,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,1,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,5.072605
5,121,Fullmetal Alchemist,0.821453,1.816414,10.539721,1,1,0,1,0,0,1,1,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,5.165042
6,3588,Soul Eater,0.821453,1.570345,10.173871,1,1,0,1,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,1,5.219682
7,6547,Angel Beats!,0.012218,1.875470,12.666215,1,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,5.236739
8,4224,Toradora!,0.267766,1.934527,11.145239,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,5.290223
9,22319,Tokyo Ghoul,-0.009078,1.560502,10.859786,1,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,1,5.401692


In [26]:
with open('anime.pkl', 'wb') as file:
        pkl.dump(df, file)